# Test 6 (paper) - Experimento 6 (PT)

Entrenamiento de Mask R-CNN con dataset modificado para emular "Test 6"

### Hiperparametros
* **epoch = 100**
    * steps x epoch = 146 (lotes de imagenes)
    * batch = 2
* optimizador = SGD
* Funcion de perdida = SMOOTHL1LOSS
* Metrica de evaluacion = mAP (IoU >= 0.5)
* Mini-mask shape: 56x56
* RPN anchor scales: (32, 64, 128, 256, 512)
* Tasa de aprendizaje: 0.001
* imagenes = 292
    * entrenamiento 81% = 237
    * validacion 19% = 55
* etiquetas = 9140
* **resolucion = 1024 x 800**
* etiquetas = bounding box formato VOC XML
* **numero de clases = 2 (arandano, arandano-maduro)**
* **data augmentation = true**


## Comprobar directorio principal

In [1]:
!pwd && ls -l

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport/model-training
total 2088
drwxr-xr-x 4 root root    4096 Aug  7 06:10 build
drwxr-xr-x 2 root root    4096 Aug  7 06:10 dist
drwxr-xr-x 2 root root    4096 Aug  8 03:15 mask_rcnn.egg-info
-rw-r--r-- 1 root root  163205 Sep 13 20:29 master-test_5.ipynb
-rw-r--r-- 1 root root  350337 Sep 15 00:38 master-test_5_fase_2.ipynb
-rw-r--r-- 1 root root  110478 Sep 13 20:29 master-test_6.ipynb
-rw-r--r-- 1 root root   21196 Sep 15 02:12 master-test_6_fase_2.ipynb
-rw-r--r-- 1 root root 1395023 Sep 13 01:41 master-v2.ipynb
-rw-r--r-- 1 root root   71399 Aug 17 21:33 master.ipynb
drwxr-xr-x 4 root root    4096 Sep 13 07:49 mrcnn


# Importar bibliotecas

In [2]:
# bibliotecas basicas
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# bibliotecas avanzadas 
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# bibliotecas mask rcnn 
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# biblioteca matplotlib 
import matplotlib.pyplot as plt

# bibliotecas numpy 
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# bibliotecas keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# ignorar alertas de elementos que seran descontinuados
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
#plt.show()

import imgaug.augmenters as iaa

# Fase 2 - Entrenamiento con dos clases y etiquetas de Bounding Box

En este entremamiento se utiliza un conjunto de datos simple con imágenes etiquetadas con cuadros delimitadores y una clase llamada 'Daño'. En la siguiente sección se encuentra el código para el entrenamiento del modelo. Se incluyen comentarios para describir mejor el flujo del programa.

In [3]:
class DatasetArandanos(Dataset):
    
    # load_dataset function is used to load the train and test dataset
    def load_dataset(self, dataset_dir, is_train=True):
        
        # we use add_class for each class in our dataset and assign numbers to them. 0 is background
        # self.add_class('source', 'class id', 'class name')
        self.add_class("dataset", 1, "arandano")
        self.add_class("dataset", 2, "arandano-maduro")
        #self.add_class("dataset", 3, "Level-3")
        #self.add_class("dataset", 4, "Level-4")
        
        # we concatenate the dataset_dir with /images and /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train will be true if we our training our model and false when we are testing the model
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4] # used to skip last 4 chars which is '.jpg' (class_id.jpg)
            
            # if is_train is True skip all images with id greater than and equal to 160
             # roughly 80% of dataset for training
            if is_train and int(image_id) >= 11074 :
                #print("image_id: ", image_id)
                continue
             
            # if is_train is not True skip all images with id less than 420
            if not is_train and int(image_id) < 11074:
                continue
            
            # img_path and ann_path variables are defined
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # using add_image function we pass image_id, image_path and ann_path so that the current
            # image is added to the dataset for training
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # function used to extract bouding boxes from annotated files
    def extract_boxes(self, filename):

        # you can see how the images are annotated we extracrt the width, height and bndbox values

        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>

        # </annotation>
        
        # used to parse the .xml files
        tree = ElementTree.parse(filename)
        
        # to get the root of the xml file
        root = tree.getroot()
        
        # we will append all x, y coordinated in boxes
        # for all instances of an onject
        boxes = list()
        
        # we find all attributes with name bndbox
        # bndbox will exist for each ground truth in an image
        for box in root.findall('.//object'):
            name = box.find('name').text
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
            
            # I have included this line to skip any un-annotated images
            if name=='arandano' or name=='arandano-maduro':
                boxes.append(coors)

        # extract width and height of the image
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # return boxes-> list, width-> int and height-> int 
        return boxes, width, height
    
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances        
    def load_mask(self, image_id):
        
        # info points to the current image_id
        info = self.image_info[image_id]
        
        # we get the annotation path of image_id which is dataset_dir/annots/image_id.xml
        path = info['annotation']
        
        # we call the extract_boxes method(above) to get bndbox from .xml file
        boxes, w, h = self.extract_boxes(path)
        
        # we create len(boxes) number of masks of height 'h' and width 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = list()
        
        # we loop over all boxes and generate masks (bndbox mask) and class id for each instance
        # masks will have rectange shape as we have used bndboxes for annotations
        # for example: if 2.jpg have four objects we will have following masks and class_ids
        # 000000000 000000000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000000000 022200000 000000000 000000000
        #    1         2          3         1<- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            
            # box[4] will have the name of the class for a particular damage
            if (box[4] == 'arandano'):
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('arandano'))
            elif(box[4] == 'arandano-maduro'):
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('arandano-maduro')) 
                
        # return masks and class_ids as array
        return masks, asarray(class_ids, dtype='int32')
    
    # this functions takes the image_id and returns the path of the image
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

# damage configuration class, you can change values of hyper parameters here
class ConfigArandanos(Config):

    # nombre de la configuracion
    NAME = "arandano_cfg_test_6_fase_2_"    
    
    # clase arandano + clase background + 4 clases
    NUM_CLASSES = 1 + 2
    
    # pasos por epoch y confianza minima    # STEPS_PER_EPOCH = cantidad de lotes/batchs
    #STEPS_PER_EPOCH = 73   # por epoch se entrenaran 73 lotes de 4 imagenes, dataset = 292
    STEPS_PER_EPOCH = 146
    
    # tasa de aprendizaje y momentum
    LEARNING_RATE=0.001
    LEARNING_MOMENTUM = 0.8
    
    # penalización de regularización
    WEIGHT_DECAY = 0.0001
    
    # el tamaño de la imagen está controlado por este parámetro
    IMAGE_MIN_DIM = 512
    
    # pasos de validación
    VALIDATION_STEPS = 50
    
    # número de regiones de interés generadas por imagen
    Train_ROIs_Per_Image = 200
    
    # escala de anclas RPN y proporciones (ratios) para encontrar la ROI
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)    # Longitud del lado del ancla cuadrada, en píxeles 
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]   # Proporciones de anclas por cada celda (ancho/alto). Un valor de 1 representa un ancla cuadrada y 0,5 es un ancla ancha 

    #DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0    
    DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

    IMAGES_PER_GPU = 2
    
ConfigArandanos().display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DEVICE                         /gpu:0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.8
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE 

### Entrenamiento

In [4]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [5]:
#pesos = 'mask_rcnn_coco.h5'
#pesos = 'mask_rcnn_damage_cfg_0049.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
pesos = 'arandano_cfg_test_5_fase_2_20220913T0749/mask_rcnn_arandano_cfg_test_5_fase_2__0300.h5'

conjunto_datos = 'customImages/test_6_fase_2'


In [7]:
# cargar dataset de entrenamiento
train_set = DatasetArandanos()
train_set.load_dataset(conjunto_datos, is_train=True)
train_set.prepare()

# cargar dataset de test 
test_set = DatasetArandanos()
test_set.load_dataset(conjunto_datos, is_train=False)
test_set.prepare()

# preparar la configuración llamando a la clase de configuración definida por el usuario
config = ConfigArandanos()

# definir el modelo
with tf.device(config.DEVICE):
    model = MaskRCNN(mode='training', model_dir='./', config=config)

# cargar pesos del modelo 
weights_path = pesos

# cargar los pesos del modelo
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

#augmentation = iaa.Sequential(iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sometimes(5/6,iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sequential([iaa.Fliplr(0.5), iaa.Flipud(0.5)])
#augmentation = iaa.Sometimes(5/6, iaa.Fliplr(0.5), iaa.Flipud(0.5))
augmentation = iaa.Sometimes(0.5, iaa.Fliplr(0.5), iaa.Flipud(0.5))

# start the training of model
# you can change epochs and layers (head or all)
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=100, layers='all', augmentation=augmentation)



Starting at epoch 0. LR=0.001

Checkpoint Path: ./arandano_cfg_test_6_fase_2_20220915T0216/mask_rcnn_arandano_cfg_test_6_fase_2__{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (C

/usr/local/lib/python3.8/dist-packages/imgaug/augmenters/base.py:49: SuspiciousSingleImageShapeWarning: You provided a numpy array of shape (1024, 1024, 36) as a single-image augmentation input, which was interpreted as (H, W, C). The last dimension however has a size of >=32, which indicates that you provided a multi-image array with shape (N, H, W) instead. If that is the case, you should use e.g. augmenter(imageS=<your input>) or augment_imageS(<your input>). Otherwise your multi-image input will be interpreted as a single image during augmentation.
  ia.warn(
/usr/local/lib/python3.8/dist-packages/imgaug/augmenters/base.py:49: SuspiciousSingleImageShapeWarning: You provided a numpy array of shape (1024, 1024, 78) as a single-image augmentation input, which was interpreted as (H, W, C). The last dimension however has a size of >=32, which indicates that you provided a multi-image array with shape (N, H, W) instead. If that is the case, you should use e.g. augmenter(imageS=<your inpu

Epoch 1/100


/usr/local/lib/python3.8/dist-packages/imgaug/augmenters/base.py:49: SuspiciousSingleImageShapeWarning: You provided a numpy array of shape (1024, 1024, 44) as a single-image augmentation input, which was interpreted as (H, W, C). The last dimension however has a size of >=32, which indicates that you provided a multi-image array with shape (N, H, W) instead. If that is the case, you should use e.g. augmenter(imageS=<your input>) or augment_imageS(<your input>). Otherwise your multi-image input will be interpreted as a single image during augmentation.
  ia.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("t

/usr/local/lib/python3.8/dist-packages/imgaug/augmenters/base.py:49: SuspiciousSingleImageShapeWarning: You provided a numpy array of shape (1024, 1024, 52) as a single-image augmentation input, which was interpreted as (H, W, C). The last dimension however has a size of >=32, which indicates that you provided a multi-image array with shape (N, H, W) instead. If that is the case, you should use e.g. augmenter(imageS=<your input>) or augment_imageS(<your input>). Otherwise your multi-image input will be interpreted as a single image during augmentation.
  ia.warn(
/usr/local/lib/python3.8/dist-packages/imgaug/augmenters/base.py:49: SuspiciousSingleImageShapeWarning: You provided a numpy array of shape (1024, 1024, 78) as a single-image augmentation input, which was interpreted as (H, W, C). The last dimension however has a size of >=32, which indicates that you provided a multi-image array with shape (N, H, W) instead. If that is the case, you should use e.g. augmenter(imageS=<your inpu

...

146/146 [==============================] - 537s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 3.6531 - rpn_class_loss: 0.7378 - rpn_bbox_loss: 1.2913 - mrcnn_class_loss: 0.4902 - mrcnn_bbox_loss: 0.6517 - mrcnn_mask_loss: 0.4820 - val_loss: 2.6930 - val_rpn_class_loss: 0.6831 - val_rpn_bbox_loss: 0.6934 - val_mrcnn_class_loss: 0.3378 - val_mrcnn_bbox_loss: 0.4880 - val_mrcnn_mask_loss: 0.4908
Epoch 2/100
146/146 [==============================] - 481s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 1.8703 - rpn_class_loss: 0.2035 - rpn_bbox_loss: 0.3628 - mrcnn_class_loss: 0.4648 - mrcnn_bbox_loss: 0.3892 - mrcnn_mask_loss: 0.4500 - val_loss: 2.6785 - val_rpn_class_loss: 0.5712 - val_rpn_bbox_loss: 0.6237 - val_mrcnn_class_loss: 0.5927 - val_mrcnn_bbox_loss: 0.4436 - val_mrcnn_mask_loss: 0.4473
Epoch 3/100
146/146 [==============================] - 476s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 1.6299 - rpn_class_loss: 0.1448 - rpn_bbox_loss: 0.2977 - mrcnn_class_loss: 0.4335 -

Epoch 21/100
146/146 [==============================] - 475s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.7651 - rpn_class_loss: 0.0346 - rpn_bbox_loss: 0.1001 - mrcnn_class_loss: 0.1973 - mrcnn_bbox_loss: 0.1279 - mrcnn_mask_loss: 0.3051 - val_loss: 3.4832 - val_rpn_class_loss: 1.2943 - val_rpn_bbox_loss: 0.6929 - val_mrcnn_class_loss: 0.7717 - val_mrcnn_bbox_loss: 0.3415 - val_mrcnn_mask_loss: 0.3827
Epoch 22/100
146/146 [==============================] - 476s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.7162 - rpn_class_loss: 0.0297 - rpn_bbox_loss: 0.0842 - mrcnn_class_loss: 0.1823 - mrcnn_bbox_loss: 0.1198 - mrcnn_mask_loss: 0.3002 - val_loss: 3.6363 - val_rpn_class_loss: 1.4200 - val_rpn_bbox_loss: 0.7269 - val_mrcnn_class_loss: 0.8006 - val_mrcnn_bbox_loss: 0.3198 - val_mrcnn_mask_loss: 0.3690
Epoch 23/100
146/146 [==============================] - 472s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.7266 - rpn_class_loss: 0.0304 - rpn_bbox_loss: 0.0939 - mrcnn_class

Epoch 41/100
146/146 [==============================] - 474s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.4509 - rpn_class_loss: 0.0088 - rpn_bbox_loss: 0.0418 - mrcnn_class_loss: 0.0993 - mrcnn_bbox_loss: 0.0651 - mrcnn_mask_loss: 0.2359 - val_loss: 4.3675 - val_rpn_class_loss: 1.9576 - val_rpn_bbox_loss: 0.9446 - val_mrcnn_class_loss: 0.8130 - val_mrcnn_bbox_loss: 0.2858 - val_mrcnn_mask_loss: 0.3666
Epoch 42/100
146/146 [==============================] - 521s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.4390 - rpn_class_loss: 0.0094 - rpn_bbox_loss: 0.0410 - mrcnn_class_loss: 0.0949 - mrcnn_bbox_loss: 0.0628 - mrcnn_mask_loss: 0.2308 - val_loss: 5.4474 - val_rpn_class_loss: 2.5296 - val_rpn_bbox_loss: 1.1550 - val_mrcnn_class_loss: 1.0270 - val_mrcnn_bbox_loss: 0.2896 - val_mrcnn_mask_loss: 0.4463
Epoch 43/100
146/146 [==============================] - 491s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.4307 - rpn_class_loss: 0.0092 - rpn_bbox_loss: 0.0425 - mrcnn_class

Epoch 61/100
146/146 [==============================] - 424s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.3287 - rpn_class_loss: 0.0051 - rpn_bbox_loss: 0.0284 - mrcnn_class_loss: 0.0631 - mrcnn_bbox_loss: 0.0427 - mrcnn_mask_loss: 0.1894 - val_loss: 4.1058 - val_rpn_class_loss: 1.7438 - val_rpn_bbox_loss: 0.8847 - val_mrcnn_class_loss: 0.8413 - val_mrcnn_bbox_loss: 0.2595 - val_mrcnn_mask_loss: 0.3765
Epoch 62/100
146/146 [==============================] - 495s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.3158 - rpn_class_loss: 0.0044 - rpn_bbox_loss: 0.0276 - mrcnn_class_loss: 0.0592 - mrcnn_bbox_loss: 0.0412 - mrcnn_mask_loss: 0.1835 - val_loss: 6.2701 - val_rpn_class_loss: 3.1707 - val_rpn_bbox_loss: 1.2715 - val_mrcnn_class_loss: 1.0476 - val_mrcnn_bbox_loss: 0.2836 - val_mrcnn_mask_loss: 0.4966
Epoch 63/100
146/146 [==============================] - 453s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.3288 - rpn_class_loss: 0.0053 - rpn_bbox_loss: 0.0310 - mrcnn_class

Epoch 81/100
146/146 [==============================] - 441s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.2490 - rpn_class_loss: 0.0024 - rpn_bbox_loss: 0.0190 - mrcnn_class_loss: 0.0473 - mrcnn_bbox_loss: 0.0295 - mrcnn_mask_loss: 0.1507 - val_loss: 5.6734 - val_rpn_class_loss: 2.8836 - val_rpn_bbox_loss: 1.1082 - val_mrcnn_class_loss: 0.9517 - val_mrcnn_bbox_loss: 0.2712 - val_mrcnn_mask_loss: 0.4587
Epoch 82/100
146/146 [==============================] - 477s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.2772 - rpn_class_loss: 0.0037 - rpn_bbox_loss: 0.0246 - mrcnn_class_loss: 0.0512 - mrcnn_bbox_loss: 0.0333 - mrcnn_mask_loss: 0.1643 - val_loss: 6.4886 - val_rpn_class_loss: 3.2608 - val_rpn_bbox_loss: 1.1411 - val_mrcnn_class_loss: 1.2387 - val_mrcnn_bbox_loss: 0.3069 - val_mrcnn_mask_loss: 0.5410
Epoch 83/100
146/146 [==============================] - 453s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.2635 - rpn_class_loss: 0.0034 - rpn_bbox_loss: 0.0220 - mrcnn_class